In [1]:
import json
import pandas as pd
import pickle
from collections import Counter, defaultdict

In [2]:
classes_5 = ['Unrelated', 'Consistent', 'Indirect inconsistency', "Factual inconsistency", 'Surface contradiction']                             

In [3]:
df_gold = pd.read_csv(r"C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\data\qualtrics_survey\results\df_gold_all.csv")

In [4]:
with open(r'C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\data\qualtrics_survey\participant_to_repeated_sample_by_class.json', 'r') as f:
    participant_to_repeated_sample_by_class = json.load(f)

In [5]:
with open(r'C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\data\qualtrics_survey\change_per_class.pkl', 'rb') as f:
    change_per_class = pickle.load(f)

In [6]:
for participant in participant_to_repeated_sample_by_class:
    for class_name in classes_5:
        total_answered_class = 0
        # only look when they switched 2 times
        total_answered_class += len(participant_to_repeated_sample_by_class[participant][class_name]['2_switches'])
        participant_to_repeated_sample_by_class[participant][class_name]['total_answered_class'] = total_answered_class
        
        # count how many time they switched
        answers_2_switches = participant_to_repeated_sample_by_class[participant][class_name]['2_switches']
        count_switched_class = [1 for el in answers_2_switches if el!= class_name]
        count_switched_class = sum(count_switched_class)
        participant_to_repeated_sample_by_class[participant][class_name]['total_switched_2_times_class'] = count_switched_class
            


In [7]:
participant_to_repeated_sample_by_class['v10']

{'total_repeated': 57,
 'Unrelated': {'2_switches': ['Unrelated'],
  '3_switches': [],
  '4_switches': [],
  'total_answered_class': 1,
  'total_switched_2_times_class': 0},
 'Consistent': {'2_switches': ['Consistent',
   'Consistent',
   'Consistent',
   'Consistent'],
  '3_switches': [],
  '4_switches': [],
  'total_answered_class': 4,
  'total_switched_2_times_class': 0},
 'Indirect inconsistency': {'2_switches': ['Indirect inconsistency',
   'Factual inconsistency',
   'Consistent',
   'Indirect inconsistency',
   'Indirect inconsistency',
   'Consistent',
   'Factual inconsistency',
   'Indirect inconsistency'],
  '3_switches': [],
  '4_switches': [],
  'total_answered_class': 8,
  'total_switched_2_times_class': 4},
 'Factual inconsistency': {'2_switches': ['Factual inconsistency',
   'Surface contradiction',
   'Surface contradiction',
   'Consistent',
   'Factual inconsistency',
   'Indirect inconsistency',
   'Indirect inconsistency',
   'Factual inconsistency',
   'Indirect i

In [7]:
# how much they shifted by class - 2 times switch
switches_2_times = dict()

switches_2_times['Unrelated'] = {'v10': 9, 'v3': 4, 'v5': 4, 'v6': 3, 'v7': 3, 'v4': 2, 'v8': 2, 'v1': 1}
switches_2_times['Consistent'] = {'v10': 8, 'v3': 4, 'v5': 4, 'v6': 3, 'v7': 3, 'v4': 2, 'v8': 2, 'v1': 1}
switches_2_times['Indirect inconsistency'] = {'v10': 5, 'v5': 4, 'v3': 2, 'v6': 2, 'v4': 2, 'v1': 1, 'v7': 1, 'v8': 1}
switches_2_times['Factual inconsistency'] = {'v10': 6, 'v3': 4, 'v7': 2, 'v4': 1, 'v6': 1, 'v8': 1}
switches_2_times['Surface contradiction'] = {'v10': 8, 'v5': 4, 'v6': 3, 'v7': 3, 'v3': 2, 'v8': 2, 'v1': 1, 'v4': 1}

### Ignore everything above

In [5]:
with open(r'C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\data\qualtrics_survey\question_repeated_trial.pkl', 'rb') as f:
    question_repeated_trial = pickle.load(f)

In [17]:
change_per_class = dict()
# divide into groups. answered 2 times, 3 times, 4 times etc.
change_per_class['2_times'] = dict()

for num_times in change_per_class:
    # for each number of switches, for each class create a dict
    for class_name in ['Unrelated', 'Consistent', 'Indirect inconsistency', 'Factual inconsistency', 'Surface contradiction']:
        change_per_class[num_times][class_name] = {}
        change_per_class[num_times][class_name]['answers'] = []
        change_per_class[num_times][class_name]['annotators'] = []
        
        
change_per_annotator = dict()
for i in range(1, 11, 1):
    change_per_annotator[f'v{i}'] = dict()
    for class_name in classes_5:
        change_per_annotator[f'v{i}'][class_name] = dict()
        change_per_annotator[f'v{i}']['total_repeated_questions'] = 0
        change_per_annotator[f'v{i}']['total_switches'] = 0
        change_per_annotator[f'v{i}'][class_name]['repeated_class'] = 0
        change_per_annotator[f'v{i}'][class_name]['switched'] = 0


In [19]:
# go through each sample which was repeatedly annotated
for el in question_repeated_trial:
    # look at each annotator
    for annotator in question_repeated_trial[el]:
        # their answers
        answers = question_repeated_trial[el][annotator]
        # this time we ignore how many times they switched, just look at the second answer.
        
        change_per_class['2_times'][answers[0]]['answers'].append(answers[1])
        change_per_class['2_times'][answers[0]]['annotators'].append(annotator)
        
        # we only add 1 becuase we consider only the first switch    
        change_per_annotator[annotator]['total_repeated_questions'] += 1
        change_per_annotator[annotator][answers[0]]['repeated_class'] += 1
        if answers[0] != answers[1]:
            change_per_annotator[annotator]['total_switches'] += 1
            change_per_annotator[annotator][answers[0]]['switched'] += 1
    

In [21]:
change_per_annotator

{'v1': {'Unrelated': {'repeated_class': 3, 'switched': 1},
  'total_repeated_questions': 18,
  'total_switches': 6,
  'Consistent': {'repeated_class': 4, 'switched': 1},
  'Indirect inconsistency': {'repeated_class': 6, 'switched': 2},
  'Factual inconsistency': {'repeated_class': 1, 'switched': 0},
  'Surface contradiction': {'repeated_class': 4, 'switched': 2}},
 'v2': {'Unrelated': {'repeated_class': 4, 'switched': 0},
  'total_repeated_questions': 18,
  'total_switches': 1,
  'Consistent': {'repeated_class': 2, 'switched': 0},
  'Indirect inconsistency': {'repeated_class': 5, 'switched': 0},
  'Factual inconsistency': {'repeated_class': 0, 'switched': 0},
  'Surface contradiction': {'repeated_class': 7, 'switched': 1}},
 'v3': {'Unrelated': {'repeated_class': 1, 'switched': 0},
  'total_repeated_questions': 30,
  'total_switches': 5,
  'Consistent': {'repeated_class': 2, 'switched': 0},
  'Indirect inconsistency': {'repeated_class': 3, 'switched': 1},
  'Factual inconsistency': {'r

now calculate percentage of times when people shifted per each class

In [26]:
for class_name in change_per_class['2_times']:
    print(class_name)
    answers = change_per_class['2_times'][class_name]['answers']
    annotators = change_per_class['2_times'][class_name]['annotators']

    # Step 1: Initialize a dictionary to track switches
    switch_count = {}

    # Step 2: Iterate over the data and count non-"Factual inconsistency" answers
    for annotator, answer in zip(annotators, answers):
        if answer != class_name:  # Check if it's a "switch"
            switch_count[annotator] = switch_count.get(annotator, 0) + 1

    # Step 3: Display the results
    sorted_switch_count = dict(sorted(switch_count.items(), key=lambda item: item[1], reverse=True))
    
    sorted_switch_percentage = dict()
    for annotator in sorted_switch_count:
        sorted_switch_percentage[annotator] = sorted_switch_count[annotator]/change_per_annotator[annotator][class_name]['repeated_class']
    # calculate percentage
    print(sorted_switch_count)
    print(sorted_switch_percentage)

Unrelated
{'v4': 3, 'v1': 1, 'v7': 1, 'v8': 1}
{'v4': 0.75, 'v1': 0.3333333333333333, 'v7': 0.5, 'v8': 0.16666666666666666}
Consistent
{'v4': 2, 'v1': 1, 'v9': 1}
{'v4': 0.2857142857142857, 'v1': 0.25, 'v9': 0.25}
Indirect inconsistency
{'v5': 4, 'v10': 4, 'v1': 2, 'v6': 1, 'v3': 1, 'v4': 1, 'v7': 1}
{'v5': 0.26666666666666666, 'v10': 0.5, 'v1': 0.3333333333333333, 'v6': 0.2, 'v3': 0.3333333333333333, 'v4': 0.5, 'v7': 0.08333333333333333}
Factual inconsistency
{'v10': 7, 'v3': 4, 'v7': 2, 'v4': 1, 'v6': 1, 'v9': 1, 'v8': 1}
{'v10': 0.6363636363636364, 'v3': 1.0, 'v7': 0.6666666666666666, 'v4': 0.3333333333333333, 'v6': 0.25, 'v9': 1.0, 'v8': 0.5}
Surface contradiction
{'v7': 4, 'v8': 3, 'v1': 2, 'v5': 2, 'v9': 1, 'v2': 1, 'v6': 1}
{'v7': 0.5714285714285714, 'v8': 1.0, 'v1': 0.5, 'v5': 0.4, 'v9': 0.16666666666666666, 'v2': 0.14285714285714285, 'v6': 0.2}


now need also totals for each annotator

In [27]:
total_switches_per_class = dict()
for class_name in classes_5:
    total_switches_per_class[class_name] = dict()
    total_switches_per_class[class_name]['total_switch'] = 0
    total_switches_per_class[class_name]['total_repeated'] = 0

for class_name in classes_5:
    for annotator in change_per_annotator:
        total_switches_per_class[class_name]['total_switch'] += change_per_annotator[annotator][class_name]['switched']
        total_switches_per_class[class_name]['total_repeated'] += change_per_annotator[annotator][class_name]['repeated_class']

In [28]:
total_switches_per_class

{'Unrelated': {'total_switch': 6, 'total_repeated': 32},
 'Consistent': {'total_switch': 4, 'total_repeated': 36},
 'Indirect inconsistency': {'total_switch': 14, 'total_repeated': 70},
 'Factual inconsistency': {'total_switch': 17, 'total_repeated': 34},
 'Surface contradiction': {'total_switch': 14, 'total_repeated': 71}}

In [29]:
for class_name in total_switches_per_class:
    print(class_name, ":", total_switches_per_class[class_name]['total_switch']/total_switches_per_class[class_name]['total_repeated'])

Unrelated : 0.1875
Consistent : 0.1111111111111111
Indirect inconsistency : 0.2
Factual inconsistency : 0.5
Surface contradiction : 0.19718309859154928


Now analyse how often model shifted in its judgements

In [31]:
df_gold['ChatGPT-4 5 runs'][0]

"['Indirect inconsistency', 'Indirect inconsistency', 'Indirect inconsistency', 'Indirect inconsistency', 'Indirect inconsistency']"